# ODIN Kaggle - Train on CIFAR-10 and Test on SVHN Cropped

This notebook is set up to run on Kaggle since they have free GPU hours. At a high level, this notebook
- Clones my (Harry's) forked repo of ODIN since their code has some bugs in Python 3 (I'm assuming their syntax was valid in Python 2 or something)
- Downloads their DenseNet-BC trained on CIFAR-10
- Downloads SVHN cropped test set
- Evaluates the model using ODIN and using the baseline

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

: 

In [1]:
!git clone https://github.com/harryli0088/odin

Cloning into 'odin'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 41 (delta 10), reused 32 (delta 10), pack-reused 0
Unpacking objects: 100% (41/41), 486.16 KiB | 6.85 MiB/s, done.


In [2]:
# download dense net model
%cd /kaggle/working/odin/models
!wget https://www.dropbox.com/s/wr4kjintq1tmorr/densenet10.pth.tar.gz

/kaggle/working/odin/models
--2022-11-15 02:33:10--  https://www.dropbox.com/s/wr4kjintq1tmorr/densenet10.pth.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/wr4kjintq1tmorr/densenet10.pth.tar.gz [following]
--2022-11-15 02:33:10--  https://www.dropbox.com/s/raw/wr4kjintq1tmorr/densenet10.pth.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6e6c60dde3ef38fae373c89199.dl.dropboxusercontent.com/cd/0/inline/BwxgPIjJnZXszf_FFgLV_9ZlCaLbmJquCR-PIULt4ivhqRKNRpowTl-npGiVYSK1o2Xe7GBBfK_f0CATKZNunso_Rf8PEpVgUUBqxCGB37uesq0ffrNItutmBVDY4JC_jx6XhYWUsw9c0eaxX8cJrJg09sJWio77cA6AbbNnDCELwg/file# [following]
--2022-11-15 02:33:11--  https://uc6e6c60dde3ef38fae373c89199.dl.dropboxusercontent.com/cd/0/inline/BwxgPIjJnZXszf_FFgLV_9ZlC

In [3]:
%cd /kaggle/working/odin/models
!tar -xf densenet10.pth.tar.gz
!ls

/kaggle/working/odin/models
densenet10.pth	densenet10.pth.tar.gz


In [4]:
# download SVHN cropped test images
!mkdir /kaggle/working/odin/data
!mkdir /kaggle/working/odin/data/SVHN_Cropped
!mkdir /kaggle/working/odin/data/SVHN_Cropped/test
%cd /kaggle/working/odin/data
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat
!ls

/kaggle/working/odin/data
--2022-11-15 02:33:25--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  16.9MB/s    in 3.6s    

2022-11-15 02:33:29 (16.9 MB/s) - ‘test_32x32.mat’ saved [64275384/64275384]

SVHN_Cropped  test_32x32.mat


In [5]:
# unpack SVHN images
from scipy.io import loadmat
import matplotlib.pyplot as plt

svhn = dict()
loadmat('./test_32x32.mat', mdict=svhn)
num_images = svhn["X"].shape[3]
print("num_images",num_images)
for i in range(num_images):
    plt.imsave("./SVHN_Cropped/test/{}.png".format(i),svhn["X"][:,:,:,i])
    # plt.imshow(svhn["X"][:,:,:,0])
    # plt.show()

num_images 26032


In [6]:
#verify SVHN test images were downloaded
!ls -U SVHN_Cropped/test | head -5

8933.png
13242.png
17356.png
2115.png
16866.png
ls: write error: Broken pipe


In [7]:
%cd /kaggle/working/odin/code
# from the ODIN README:
# model: DenseNet-BC, in-distribution: CIFAR-10, out-distribution: TinyImageNet (crop)
# magnitude: 0.0014, temperature 1000, gpu: 0
!python main.py --nn densenet10 --out_dataset SVHN_Cropped --magnitude 0.0014 --temperature 1000 --gpu 0

/kaggle/working/odin/code
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the o